In [1]:
import os
from langchain.llms import CTransformers
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.conversation.memory import ConversationBufferMemory
from ctransformers import AutoModelForCausalLM
from transformers import AutoTokenizer
from langchain.llms  import LlamaCpp
from langchain.callbacks.manager import CallbackManager

c:\Users\LENOVO\sagar\work\AI_ML\gitrepo\AIML\GenAI\YOUTUBES\Talk_with_your_data\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:

llama2_model_path = "model/llama-2-7b-chat.ggmlv3.q8_0.bin"

In [7]:
llm=CTransformers(
        model=llama2_model_path,
        model_type="llama",
        config={'max_new_tokens':128,
                'temperature':0.01}
)

RepositoryNotFoundError: 404 Client Error. (Request ID: Root=1-6606d20f-734c94aa4d1e8ca451a08f00;8fd6be26-828d-4648-b2d5-972d9a4f0f70)

Repository Not Found for url: https://huggingface.co/api/models/model/llama-2-7b-chat.ggmlv3.q8_0.bin/revision/main.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.